In [60]:
import pandas as pd
import numpy as np
import scipy
import os


import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

from datetime import date
import seaborn as sns
import re
from datetime import datetime

# fuzz is used to compare TWO strings
from fuzzywuzzy import fuzz

# process is used to compare a string to MULTIPLE other strings
from fuzzywuzzy import process

<h1 style="text-align:center;font-family: Arial; font-size:2.0em;color:black;"> GENERATION OF NEW DATA </h1>
<br/>
<a style="font-size: 30px" href="https://mockaroo.com">https://mockaroo.com/</a>

<h1 style="text-align:center;font-family: Arial; font-size:2.4em;color:black;"> VIEWING INITIAL DATA </h1>

In [64]:
canonico = pd.read_csv('D:\\documentos\\apresentacoes\\input.csv')
duplicates = pd.read_csv('D:\\documentos\\apresentacoes\\output.csv')

In [65]:
duplicates.head()

,id,first_name,last_name,email,gender,country,street
0,8,Reg,Balsom,rbalsom7.@foxnews.com,Male,Sweden,75124
1,8,Reg Balsom,Balsom,rbalsom72@foxnews.com,Male,Sweden,75124
2,10,Charil,Fishenden,cfishenden95@shareasale.com,Female,Indonesia,87627
3,10,Charil,Fishenden,cfishenden9_@shareasale.com,Female,Indonesia,87627
4,10,Charil Fishenden,Fishenden,cfishenden9-@shareasale.com,Female,Indonesia,87627


In [66]:
canonico.head()

,id,first_name,last_name,email,gender,country,street
0,1,Reinold,Easom,reasom0@seesaa.net,Male,United States,6660
1,2,Guntar,Esche,gesche1@google.com.br,Male,Argentina,98016
2,3,Wernher,Mingauld,wmingauld2@wikimedia.org,Male,Czech Republic,1543
3,4,Vladamir,Retchford,vretchford3@soundcloud.com,Male,Indonesia,965
4,5,Godart,Tynewell,gtynewell4@prnewswire.com,Male,Spain,6


In [67]:
print("Shape canonico : {}".format(canonico.shape))
print("Shape duplicates : {}".format(duplicates.shape))

Shape canonico : (1000, 7)
Shape duplicates : (485, 7)


<h1 style="text-align:center;font-family: Arial; font-size:2.0em;color:black;"> CHECK SIMILAR CUSTOMERS IN THE DATAFRAME DUPLICATES </h1>

In [68]:
#check for similar customers
def get_ratio(row): 
    email         = fuzz.token_set_ratio(duplicates['email'], row['email'])
    last_name     = fuzz.token_set_ratio(duplicates['last_name'], row['last_name'])
    first_name    = fuzz.token_set_ratio(duplicates['first_name'], row['first_name'])
    
    weights = [5.0, 2.5, 2.5]
    
    return np.average(np.array([email,last_name,first_name]), weights=weights) 

In [69]:
canonico['SCORE']  = canonico.apply(get_ratio, axis=1)

In [70]:
canonico.sort_values('SCORE',ascending=False).head(50).style.background_gradient(subset=['SCORE'],cmap='summer')

,id,first_name,last_name,email,gender,country,street,SCORE
973,974,Emlynn,Bentje,ebentjer1@posterous.com,Female,Philippines,3635,100
7,8,Reg,Balsom,rbalsom7@foxnews.com,Male,Sweden,75124,100
975,976,Clerc,Beer,cbeerr3@fotki.com,Male,Portugal,6,100
15,16,Forest,Southerns,fsouthernsf@blogs.com,Male,Spain,73940,100
970,971,Cathrin,Beddall,cbeddallqy@sina.com.cn,Female,France,178,100
953,954,Farley,MacMenamie,fmacmenamieqh@rakuten.co.jp,Male,Sweden,8,100
9,10,Charil,Fishenden,cfishenden9@shareasale.com,Female,Indonesia,87627,100
963,964,Lowe,Matisse,lmatisseqr@gravatar.com,Male,China,50,100
73,74,Rube,Petre,rpetre21@mtv.com,Male,Uganda,2551,100
955,956,Sander,Gloster,sglosterqj@sciencedirect.com,Male,Portugal,646,100


In [71]:
process.extract("lmatisseqr@gravatar.com", duplicates.email,scorer=fuzz.token_sort_ratio)

[('lmatisseqr.@gravatar.com', 100, 462),
 ('lmatisseqr_@gravatar.com', 98, 461),
 ('lmatisseqr2@gravatar.com', 98, 463),
 ('lmatisseqr2@gravatar.com', 98, 464),
 ('bliffent2@gravatar.com', 53, 397)]

In [72]:
process.extract("wwraithjp@sciencedirect.com", duplicates.email,scorer=fuzz.token_sort_ratio)

[('sglosterqj-@sciencedirect.com', 73, 460),
 ('gkinnaneex2@sciencedirect.com', 61, 239),
 ('gkinnaneex2@sciencedirect.com', 61, 240),
 ('gkinnaneex_@sciencedirect.com', 61, 241),
 ('spitts226@sciencedaily.com', 60, 24)]

In [73]:
process.extract("cbeerr3@fotki.com", duplicates.email,scorer=fuzz.token_sort_ratio)

[('cbeerr3-@fotki.com', 100, 475),
 ('cbeerr34@fotki.com', 97, 474),
 ('cbeerr31@fotki.com', 97, 476),
 ('cbeerr36@fotki.com', 97, 477),
 ('aleversha131@wix.com', 54, 11)]

<h2 style="text-align:center;font-family: Arial; font-size:2.4em;color:black;"> CONCAT CANONIC DATA WITH DUPLICATES </h2>

In [74]:
df = pd.concat([canonico,duplicates])

In [75]:
process.extract("cfishenden95@shareasale.com", df.email,scorer=fuzz.token_sort_ratio)

[('cfishenden95@shareasale.com', 100, 2),
 ('cfishenden9@shareasale.com', 98, 9),
 ('cfishenden9-@shareasale.com', 98, 4),
 ('cfishenden9_@shareasale.com', 96, 3),
 ('murienae@shareasale.com', 64, 374)]

<h2 style="text-align:center;font-family: Arial; font-size:2.4em;color:black;"> SCRIPT DEDUPE </h2>

In [77]:
run = True

In [78]:
#!/usr/bin/python
# -- coding: utf-8 --
"""
This code is responsible for using similar information deduplication.

    * The input file must have the follows formats .csv or .xlsx;
    * The output file has the format be the .csv;
    * The groups selected for agent review will be groups that contain different resolution types and at least one suspect;
    
"""

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore') 
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UnicodeWarning)
warnings.filterwarnings("ignore", category=Warning)
warnings.filterwarnings("ignore", category=SyntaxWarning)

from datetime import date 
from datetime import datetime

from future.builtins import next
from contextlib import contextmanager
import os
import csv
import re
import time
import logging
import optparse
import dedupe
from unidecode import unidecode
import gc 
import glob

optp = optparse.OptionParser()
optp.add_option('-v', '--verbose', dest='verbose', action='count',
                help='Increase verbosity (specify multiple times for more)'
                )

input_file = 'input\\*'
output_file = 'output\\output_auto.csv'
settings_file = 'confg\\auto_learned_settings'
training_file = 'confg\\auto_training.json' 
NEW_PATH_FILES = datetime.now().strftime('%Y_%m_%d_%H_%M')

_fields = ['id', 'first_name', 'last_name', 'email', 'gender', 'country',
       'street']

def createRequiredFolders():
    """
        Method responsible for creating required folders
    """

    createDir('output\\')
    createDir('confg\\')
    createDir('clean\\')
    createDir('input\\')

def readDataDF(path):
    """ Read the dataset found at the specified path
    
    Arguments:
        path {str} -- FILE PATH 
    
    Raises:
        Exception: DATASET NOT FOUND
        Exception: ONLY ONE INPUT FILE IS RECOMMENDED
        Exception: DATASET NOT FOUND
    
    Returns:
        dict -- DICT WITH A DATAFRAME AND THE PATH
    """ 

    all_files = [file for file in glob.glob('{}\\*'.format(path)) if file.endswith('.csv') or file.endswith('.xlsx')] 
    
    if len(all_files) == 0 :
        raise Exception(' DATASET NOT FOUND IN FOLDER {}'.format(path))
    if len(all_files) > 1:
        raise Exception(' ONLY ONE INPUT FILE IS RECOMMENDED  ')

    if all_files[0].endswith('.csv'):
        print('==> READING CSV...')
        return  pd.read_csv(all_files[0]) , all_files[0]
    if all_files[0].endswith('.xlsx'):
        print('==> READING EXCEL...')
        return  pd.read_excel(all_files[0]) , all_files[0]

    raise Exception(' DATASET NOT FOUND ')


@contextmanager
def timer(title):
    """
        count time
    """

    t0 = time.time()
    yield
    print("*** {} - DONE IN {:.0f}s ***".format(title, time.time() - t0))
    
def createDir(directory):
    """ CREATE DIRECTORY IF NECESSARY
    
    Arguments:
        directory {str} -- PATH OF THE NEW DIRECTORY
    
    Returns:
        str -- PATH OF THE NEW DIRECTORY
    """

    if not os.path.exists(directory):
        print('*** CREATING DIRECTORY => {} ***'.format(directory))
        os.makedirs(directory)
    return directory

def deleteFile(filePath):
    """ DELETE DIRECTORY IF EXISTS
    
    Arguments:
        filePath {str} -- PATH OF DIRECTORY TO BE DELETED
    """

    if os.path.exists(filePath):
        try:
            print("DELETING FILE {}".format(filePath))
            os.remove(filePath)
        except:
            print("ERROR WHILE DELETING FILE ", filePath)
    else:
        print("CAN NOT DELETE THE FILE AS IT DOESN'T EXISTS : {}".format(filePath))
 

def preProcess(column):
    """
        Do a little bit of data cleaning with the help of Unidecode and Regex.
        Things like casing, extra spaces, quotes and new lines can be ignored.
    """
    try : # python 2/3 string differences
        column = column.decode('utf8')
    except AttributeError:
        pass
    column = unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    # If data is missing, indicate that by setting the value to `None`
    if not column:
        column = None
    return column


def readData(filename):
    """
        Read in our data from a CSV file and create a dictionary of records, 
        where the key is a unique record ID and each value is dict
    """

    data_d = {}
    with open(filename, encoding="utf8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
            row_id = int(row['ID'])
            data_d[row_id] = dict(clean_row)

    return data_d
    
def hasAllColumns(data, columns=[]):
    """  Verify that the data set has the required columns
    
    Arguments:
        data {DataFrame} -- DATAFRAME FOR VALIDATION
    
    Keyword Arguments:
        columns {list} -- LIST OF COLUMNS THAT MUST EXIST IN THE DATA  (default: {[]})
    
    Raises:
        Exception: COLUMNS REQUIRED FOR PROCEDURE NOT FOUND
    """
    
    print("==> DATASET HAS ALL REQUIRED COLUMNS...")
    list_err = []
    for col in columns:
        if col not in data.columns:
            list_err.append(col)
    if len(list_err) >  0 :
        raise Exception('COLUMNS REQUIRED FOR PROCEDURE NOT FOUND: {}'.format(list_err), 'ERRO')
                    
def validationDataset(data, columns=[]):
    """ Method responsible for performing the necessary validations on the dataset
    
    Arguments:
        data {DataFrame} -- DATAFRAME FOR VALIDATION
    
    Keyword Arguments:
        columns {list} -- LIST OF COLUMNS THAT MUST EXIST IN THE DATA (default: {[]})
    
    Raises:
        Exception: DATASET IS NONE
    
    Returns:
        [DataFrame] -- MODIFIED DATAFRAME
    """


    if data is None:
        raise Exception('DATASET IS NONE' , 'ERRO')
    
    #REMOVE DATA MISSING
    print("==> REMOVE DATA MISSING (['email', 'gender', 'country','street'])...")
    data.dropna(subset=['email', 'gender', 'country','street'], how='all', inplace=True) 
    print("==> CAPITALIZE ALL COLUMNS...")
    data = data.apply(lambda x: x.astype(str).str.upper())
    #DATASET HAS ALL REQUIRED COLUMNS 
    hasAllColumns(data,columns)
    
    return data


def cleanData(data,input_file):
    """ Method responsible for proper data cleansing and validation
    
    Arguments:
        data {DataFrame} -- DATAFRAME FOR VALIDATION
        input_file {str} -- FILE INPUT PATH
    
    Returns:
        [str] -- MODIFIED FILE INPUT PATH
    """


    #validation
    data = validationDataset(data,_fields)
    df_ = data[_fields]
    df_ = df_.reset_index()
    #segunda fase
    df_ = df_.ix[:,df_.dtypes==object].apply(lambda s: s.str.replace('"', ""))
    df_ = df_.reset_index()
    #terceira fase
    df_.rename(columns={'index'    :    'ID'}, inplace=True)
    #quarta fase
    print("==> SAVING DATA..." )
    df_.to_csv(input_file.replace('input','clean').replace(' ','_').replace('.csv',"_CLEAN.csv").replace('.xlsx',"_CLEAN.csv"))
    del data, df_
    gc.collect()
    return input_file.replace('input','clean').replace(' ','_').replace('.csv',"_CLEAN.csv").replace('.xlsx',"_CLEAN.csv")

def run(input_file):
    """ Dedupe library execution
    
    Arguments:
        input_file {str} -- PATH OF FILE TO BE USED
    
    Raises:
        Exception: INPUT FILE IS NONE OR EMPTY
    """

    if input_file is None or input_file == '':
        raise Exception('INPUT FILE IS NONE OR EMPTY' , 'ERRO')

    print('==> IMPORTING DATA ...') 
    data_d = readData(input_file)  
    print('==> FINISHED IMPORTING DATA ...')
    # If a settings file already exists, we'll just load that and skip training
    if os.path.exists(settings_file):
        print('==> READING FROM', settings_file)
        with open(settings_file, 'rb') as f:
            deduper = dedupe.StaticDedupe(f)
    else:
        # ## Training

        # Define the fields dedupe will pay attention to
        fields = []
        for col in _fields:
            if col not in ['id']: 
                fields.append({'field' : col, 'type': 'String','has missing' : True})

        # Create a new deduper object and pass our data model to it.
        deduper = dedupe.Dedupe(fields)

        # If we have training data saved from a previous run of dedupe,
        # look for it and load it in.
        # __Note:__ if you want to train from scratch, delete the training_file
        if os.path.exists(training_file):
            print('reading labeled examples from ', training_file)
            with open(training_file, 'rb') as f:
                deduper.prepare_training(data_d, f)
        else:
            deduper.prepare_training(data_d)

        # ## Active learning
        # Dedupe will find the next pair of records
        # it is least certain about and ask you to label them as duplicates
        # or not.
        # use 'y', 'n' and 'u' keys to flag duplicates
        # press 'f' when you are finished
        print('==> STARTING ACTIVE LABELING...')

        dedupe.consoleLabel(deduper)

        # Using the examples we just labeled, train the deduper and learn
        # blocking predicates
        deduper.train()

        # When finished, save our training to disk
        with open(training_file, 'w') as tf:
            deduper.writeTraining(tf)

        # Save our weights and predicates to disk.  If the settings file
        # exists, we will skip all the training and learning next time we run
        # this file.
        with open(settings_file, 'wb') as sf:
            deduper.writeSettings(sf)

    # Find the threshold that will maximize a weighted average of our
    # precision and recall.  When we set the recall weight to 2, we are
    # saying we care twice as much about recall as we do precision.
    #
    # If we had more data, we would not pass in all the blocked data into
    # this function but a representative sample.

    threshold = deduper.threshold(data_d, recall_weight=1)

    # ## Clustering

    # `match` will return sets of record IDs that dedupe
    # believes are all referring to the same entity.

    print('==> CLUSTERING...')
    clustered_dupes = deduper.match(data_d, threshold)

    print('# duplicate sets', len(clustered_dupes))

    # ## Writing Results

    # Write our original data back out to a CSV with a new column called 
    # 'Cluster ID' which indicates which records refer to each other.

    cluster_membership = {}
    cluster_id = 0
    for (cluster_id, cluster) in enumerate(clustered_dupes):
        id_set, scores = cluster
        cluster_d = [data_d[c] for c in id_set]
        canonical_rep = dedupe.canonicalize(cluster_d)
        for record_id, score in zip(id_set, scores):
            cluster_membership[record_id] = {
                "CLUSTER_ID" : cluster_id,
                "canonical representation" : canonical_rep,
                "confidence": score
            }

    singleton_id = cluster_id + 1

    with open(output_file, 'w',encoding="utf8") as f_output, open(input_file,encoding="utf8") as f_input:
        writer = csv.writer(f_output)
        reader = csv.reader(f_input)

        heading_row = next(reader)
        heading_row.insert(0, 'confidence_score')
        heading_row.insert(0, 'CLUSTER_ID')
        canonical_keys = canonical_rep.keys()
        for key in canonical_keys:
            heading_row.append('canonical_' + key)

        writer.writerow(heading_row)

        for row in reader:
            row_id = int(row[0])
            if row_id in cluster_membership:
                cluster_id = cluster_membership[row_id]["CLUSTER_ID"]
                canonical_rep = cluster_membership[row_id]["canonical representation"]
                row.insert(0, cluster_membership[row_id]['confidence'])
                row.insert(0, cluster_id)
                for key in canonical_keys:
                    row.append(canonical_rep[key].encode('utf8'))
            else:
                row.insert(0, None)
                row.insert(0, singleton_id)
                singleton_id += 1
                for key in canonical_keys:
                    row.append(None)
            writer.writerow(row)
            

def runAnalysis():
    """
        Merge output data with initial data.
        Generates files with groups with fraud and different fraud status.
    """
    _fields_join_dedupe = ['id', 'first_name', 'last_name', 'email', 'gender', 'country','street']

    print('==> IMPORTING DATA ...')  
    df_dedupe = pd.read_csv(output_file)
    df  = readDataDF('input')[0]
    print('==> FINISHED IMPORTING DATA ...')
    print('==> MERGE DATA...')  
    df_final = pd.merge(df_dedupe.apply(lambda x: x.astype(str).str.upper()),df.apply(lambda x: x.astype(str).str.upper()),on=_fields, how="outer")
    print("==> REMOVE SVC_DSPCH_ID DUPLICATED... ")
    df_final = df_final.loc[df_final.drop_duplicates(keep='first').index].sort_values(by='email')
    print('==> MERGE DATA END...') 
    hasAllColumns(df_final,_fields_join_dedupe)
    print('==> CROSSTAB SUSPECT GROUPS...')  

    print('==> CREATING PATH FOR NEW FILES')
    PATH_FILES = createDir('output\\{}'.format(NEW_PATH_FILES))
    PATH_FILES_GROUPS = createDir('output\\{}\\GROUPS'.format(NEW_PATH_FILES))
    print('==> PATH CREATED SUCCESSFUL : {}'.format(PATH_FILES))

    print('==> SAVING DATA FINAL...') 
    df_final.sort_values(['CLUSTER_ID','email']).to_csv(PATH_FILES+"\\output_final_merge.csv",index=False)
    df_final.sort_values(['CLUSTER_ID','email']).groupby(['CLUSTER_ID']).apply(lambda x: saveGroupedDF(x,PATH_FILES_GROUPS))
    
    deleteFile(output_file)

    del df_final, df_dedupe, df, PATH_FILES_GROUPS, PATH_FILES
    gc.collect()
    print('==> END...') 
    
def saveGroupedDF(x,PATH_FILES_GROUPS): 
    """Method responsible for saving datafreme on the way
    
    Arguments:
        x {Series} -- data from dataframe
        PATH_FILES_GROUPS {str} -- FILE PATH 
    """
    x.to_csv(PATH_FILES_GROUPS+'\\'+ str(x.name) + '.csv',index=False)

def runDedupe( ):
    """ 
        Method responsible for the organized execution of other methods
    """
    
    start_time = datetime.now() 
    with timer("SUCCESSFUL FOLDER CREATION PROCESS"):
        print("VALIDATING IF FOLDERS ALREADY EXIST ...")
        createRequiredFolders()
    with timer("SUCCESSFUL READING PROCESS"):
        print("READING DATA, THIS MAY TAKE A FEW MINUTES. PLEASE WAIT ...")
        data, input_file = readDataDF('input')
    with timer("SUCCESSFUL CLEANING PROCESS"):
        print('DATA CLEANING STARTED ...')
        input_file = cleanData(data, input_file)
    with timer("SUCCESSFUL DEDUPE PROCESS"):
        print("DEDUPTION PROCESS STARTED...")
        run(input_file)
    with timer("SUCCESSFUL ANALYSIS PROCESS"):
        print('DATA ANALYSIS STARTED ...')        
        runAnalysis()

    print('DURATION: {}'.format(datetime.now()   - start_time))

    deleteFile(input_file)

    del input_file, data, start_time
    gc.collect()

if run :
    runDedupe()


VALIDATING IF FOLDERS ALREADY EXIST ...
*** SUCCESSFUL FOLDER CREATION PROCESS - DONE IN 0s ***
READING DATA, THIS MAY TAKE A FEW MINUTES. PLEASE WAIT ...
==> READING CSV...
*** SUCCESSFUL READING PROCESS - DONE IN 0s ***
DATA CLEANING STARTED ...
==> REMOVE DATA MISSING (['email', 'gender', 'country','street'])...
==> CAPITALIZE ALL COLUMNS...
==> DATASET HAS ALL REQUIRED COLUMNS...
==> SAVING DATA...
*** SUCCESSFUL CLEANING PROCESS - DONE IN 0s ***
DEDUPTION PROCESS STARTED...
==> IMPORTING DATA ...
==> FINISHED IMPORTING DATA ...


first_name : lorilee
last_name : madrell
email : lmadrell60@forbes.com
gender : female
country : france
street : 819

first_name : conn
last_name : pasque
email : cpasqueal@indiegogo.com
gender : male
country : russia
street : 60

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


==> STARTING ACTIVE LABELING...
n


first_name : dion
last_name : argente
email : dargenteev@diigo.com
gender : female
country : czech republic
street : 3249

first_name : debi
last_name : lawther
email : dlawtheri5@ibm.com
gender : female
country : ukraine
street : 8

0/10 positive, 1/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


...

y


first_name : clovis ham
last_name : ham
email : chamjd2@geocities.jp
gender : female
country : azerbaijan
street : 919

first_name : clovis
last_name : ham
email : chamjd6@geocities.jp
gender : female
country : azerbaijan
street : 919

11/10 positive, 5/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


f


Finished labeling
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (sameFiveCharStartPredicate, street), TfidfTextCanopyPredicate: (0.4, last_name))
INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.000100, score 0.8671930949027071
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (sameFiveCharStartPredicate, street), TfidfTextCanopyPredicate: (0.4, last_name))
INFO:dedupe.api:Maximum expected recall and precision
INFO:dedupe.api:recall: 1.000
INFO:dedupe.api:precision: 1.000
INFO:dedupe.api:With threshold: 1.000


==> CLUSTERING...
# duplicate sets 200
*** SUCCESSFUL DEDUPE PROCESS - DONE IN 356s ***
DATA ANALYSIS STARTED ...
==> IMPORTING DATA ...
==> READING CSV...
==> FINISHED IMPORTING DATA ...
==> MERGE DATA...
==> REMOVE SVC_DSPCH_ID DUPLICATED... 
==> MERGE DATA END...
==> DATASET HAS ALL REQUIRED COLUMNS...
==> CROSSTAB SUSPECT GROUPS...
==> CREATING PATH FOR NEW FILES
*** CREATING DIRECTORY => output\2020_02_05_08_35 ***
*** CREATING DIRECTORY => output\2020_02_05_08_35\GROUPS ***
==> PATH CREATED SUCCESSFUL : output\2020_02_05_08_35
==> SAVING DATA FINAL...
DELETING FILE output\output_auto.csv
==> END...
*** SUCCESSFUL ANALYSIS PROCESS - DONE IN 3s ***
DURATION: 0:05:58.805267
DELETING FILE clean\DATA_SET_CLEAN.csv


In [87]:
dedupe = readDataDF('output\\2020_02_05_08_35')[0]

==> READING CSV...


In [88]:
columns = ['CLUSTER_ID', 'confidence_score',
       'first_name', 'last_name', 'email', 'gender', 'country', 'street']

In [89]:
dedupe[columns]

,CLUSTER_ID,confidence_score,first_name,last_name,email,gender,country,street
0,0,0.999993681626506,REG,BALSOM,RBALSOM7.@FOXNEWS.COM,MALE,SWEDEN,75124
1,0,0.9999919574580142,REG BALSOM,BALSOM,RBALSOM72@FOXNEWS.COM,MALE,SWEDEN,75124
2,0,0.9999920044195776,REG,BALSOM,RBALSOM7@FOXNEWS.COM,MALE,SWEDEN,75124
3,1,0.9999942161991736,CHARIL FISHENDEN,FISHENDEN,CFISHENDEN9-@SHAREASALE.COM,FEMALE,INDONESIA,87627
4,1,0.9999953079873796,CHARIL,FISHENDEN,CFISHENDEN95@SHAREASALE.COM,FEMALE,INDONESIA,87627
5,1,0.9999942513181824,CHARIL,FISHENDEN,CFISHENDEN9@SHAREASALE.COM,FEMALE,INDONESIA,87627
6,1,0.9999953079873796,CHARIL,FISHENDEN,CFISHENDEN9_@SHAREASALE.COM,FEMALE,INDONESIA,87627
7,10,0.9999944953569628,KAYCEE,CASWALL,KCASWALL201@ALEXA.COM,FEMALE,POLAND,78
8,10,0.9999940685142198,KAYCEE CASWALL,CASWALL,KCASWALL202@ALEXA.COM,FEMALE,POLAND,78
9,10,0.9999940685142198,KAYCEE CASWALL,CASWALL,KCASWALL202@ALEXA.COM,FEMALE,POLAND,78


In [90]:
dedupe.shape

(1485, 20)

In [91]:
dedupe[dedupe.confidence_score != 'NAN'].shape

(685, 20)

In [92]:
grupos = dedupe[dedupe.confidence_score != 'NAN'].index
df = dedupe.iloc[grupos]

In [93]:
df[columns].head(100)

,CLUSTER_ID,confidence_score,first_name,last_name,email,gender,country,street
0,0,0.999993681626506,REG,BALSOM,RBALSOM7.@FOXNEWS.COM,MALE,SWEDEN,75124
1,0,0.9999919574580142,REG BALSOM,BALSOM,RBALSOM72@FOXNEWS.COM,MALE,SWEDEN,75124
2,0,0.9999920044195776,REG,BALSOM,RBALSOM7@FOXNEWS.COM,MALE,SWEDEN,75124
3,1,0.9999942161991736,CHARIL FISHENDEN,FISHENDEN,CFISHENDEN9-@SHAREASALE.COM,FEMALE,INDONESIA,87627
4,1,0.9999953079873796,CHARIL,FISHENDEN,CFISHENDEN95@SHAREASALE.COM,FEMALE,INDONESIA,87627
5,1,0.9999942513181824,CHARIL,FISHENDEN,CFISHENDEN9@SHAREASALE.COM,FEMALE,INDONESIA,87627
6,1,0.9999953079873796,CHARIL,FISHENDEN,CFISHENDEN9_@SHAREASALE.COM,FEMALE,INDONESIA,87627
7,10,0.9999944953569628,KAYCEE,CASWALL,KCASWALL201@ALEXA.COM,FEMALE,POLAND,78
8,10,0.9999940685142198,KAYCEE CASWALL,CASWALL,KCASWALL202@ALEXA.COM,FEMALE,POLAND,78
9,10,0.9999940685142198,KAYCEE CASWALL,CASWALL,KCASWALL202@ALEXA.COM,FEMALE,POLAND,78
